In [19]:
import pandas as pd
import numpy as np

In [20]:
train_policies_complete = pd.read_csv('../data/train_policies_complete.csv', index_col=0)
cols_to_drop = ['split']
train_policies_complete = train_policies_complete.drop(cols_to_drop, axis=1)

In [21]:
class DataValidation:
    def __init__(self, df):
        self._data = df
        self.raw_catvars = self._data.select_dtypes(exclude=[np.number])
        self.raw_numvars = self._data.select_dtypes(include=[np.number])

    # returns numpy array of categorical variables as they appear in dataset
    def get_categoric(self):
        return self.raw_catvars.columns.values

    # returns numpy array of numerical variables as they appear in dataset
    def get_numeric(self):
        return self.raw_numvars.columns.values

    # returns df of pct missing data points for each predictor
    def get_missing(self):
        dict_missing = {'col':[], 'pct_missing':[]}

        for col in self._data.columns:
            mean_missing = np.mean(self._data[col].isnull())
            pct_missing = round(mean_missing * 100, 5)
            
            dict_missing['col'].append(col)
            dict_missing['pct_missing'].append(pct_missing)

        df_missing = pd.DataFrame(data=dict_missing)
        return df_missing

In [22]:
train_policy = DataValidation(train_policies_complete)
for col in train_policy.get_categoric():
    print("{}: {}".format(col, train_policy._data[col].unique()))

Quote_dt: ['2015-01-28' '2018-09-03' '2016-05-18' ... '2017-06-08' '2017-04-27'
 '2017-10-02']
discount: ['Yes' 'No']
Home_policy_ind: ['Y' 'N']
state_id: ['NY' 'FL' 'MN' 'NJ' 'WI' 'CT' 'GA' 'AL']
quoted_amt: ['$5,153' '$3,090' '$14,917' ... '$6,669' '$271' '$8,428']
Prior_carrier_grp: ['Carrier_1' 'Carrier_4' 'Carrier_3' 'Carrier_5' 'Carrier_2' 'Carrier_8'
 'Carrier_6' 'Carrier_7' 'Other']
Cov_package_type: ['High' 'Medium' 'Low']
policy_id: ['policy_87209' 'policy_91413' 'policy_71845' ... 'policy_67016'
 'policy_30163' 'policy_63982']
primary_parking: ['home/driveway' 'unknown' 'parking garage' 'street']


In [25]:
train_policy.get_numeric()

array(['credit_score', 'CAT_zone', 'number_drivers', 'num_loaned_veh',
       'num_owned_veh', 'num_leased_veh', 'total_number_veh',
       'convert_ind'], dtype=object)

In [ ]:
# regex 'Quote_dt' to verify consistent format, group by year

In [ ]:
# regex 'quoted_amt' to verify consistent format, convert to int

In [ ]:
# binary encode 'discount' and 'Home_poilcy_ind'
train_policy._data['discount'] = train_policy._data['discount'].apply(lambda x: 0 if x == 'No' else 1)
train_policy._data['Home_policy_ind'] = train_policy._data['Home_policy_ind'].apply(lambda x: 0 if x == 'N' else 1)

In [ ]:
# make dummies for 'Quote_dt', 'state_id', 'prior_carrier_group', 'Cov_package_type', 'primary_parking'